# Housing Market EDA and Price Prediction Project
### Objectives:
- Import the dataset
- Inspect and clean the data set
- Understand the data (correlations, heatmap)
- Visualize data
- Data transformation
- Model development
- Model evaluation/visualization
- Model refinement

## Step 1: Import Required Libraries

In [ ]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
import os
import gc
from scipy import stats
import warnings
%matplotlib inline
warnings.filterwarnings("ignore",category=UserWarning)
warnings.filterwarnings("ignore",category=FutureWarning)
sns.set(rc={'axes.facecolor':'lightblue', 'figure.facecolor':'lightgrey'})
from tqdm import tqdm

## Step 2: Import The Dataset

In [ ]:
FILE_PATH = 'Housing.csv'

In [ ]:
raw_data = pd.read_csv(FILE_PATH)

## Step 3: Inspect And Clean The Dataset

In [ ]:
raw_data.head()

In [ ]:
raw_data.info()

In [ ]:
raw_data.describe()

In [ ]:
cols = raw_data.columns

for col in cols:
    print(col,"unique values:",raw_data[col].unique(),"\n")

### Initial inspecion of the data looks good, there are no null values. 4 fixes will be done: 
1. Cleaning headers 
2. Drop unecessary columns 
3. The conversion of yes no columns to bool 
4. Furnished status changed to ordered category

### Cleaning column headers to preferred convention

In [ ]:
headers = ['price','area','no_bedrooms','no_bathrooms','no_stories','mainroad_connection','basement','guestroom','hot_water_heating','air_conditioning','no_parking_slots','prefarea','furnishing_status']
raw_data.columns = headers
raw_data

### Dropping unecessary columns

It is unclear what the prefarea is supposet to represent. Since we cannot determine whether its the buyers preferred area, seller preferred area, or general preferred area, it will be removed from the dataframe.

In [ ]:
raw_data = raw_data.drop('prefarea',axis=1)
raw_data

### Conversion of yes no to bool

In [ ]:
yes_no_cols = ['mainroad_connection','basement','guestroom','hot_water_heating','air_conditioning']
raw_data[yes_no_cols] = raw_data[yes_no_cols].replace({'yes': True, 'no': False}).astype(bool)

### Furnished status changed to ordered category

In [ ]:
raw_data['furnishing_status'] = pd.Categorical(raw_data['furnishing_status'], categories=['unfurnished', 'semi-furnished', 'furnished'], ordered=True)

## Step 4: Understand the Data (Correlations, Heatmap)

### Numerical Feature Correlation

In [ ]:
from scipy.stats import pearsonr

numerical_columns = ['price', 'area', 'no_bedrooms', 'no_bathrooms', 'no_stories']

results = []

for feature in numerical_columns:
    if feature != 'price':  
        correlation, p_value = pearsonr(raw_data_encoded['price'], raw_data_encoded[feature])
        results.append({'Feature': feature, 'Correlation': correlation, 'P-value': p_value})

results_df = pd.DataFrame(results)

results_df.sort_values(by=['Correlation','P-value'],ascending=False)

### Bool Feature Correlation

In [ ]:
from scipy.stats import pointbiserialr

target = 'price'
features = ['mainroad_connection','basement','guestroom','hot_water_heating','air_conditioning']

results = []

for feature in features:
    correlation, p_value = pointbiserialr(raw_data[feature], raw_data[target])
    results.append({'Feature': feature, 'Correlation': correlation, 'P-value': p_value})

correlation_results = pd.DataFrame(results)
correlation_results.sort_values(by=['Correlation', 'P-value'],ascending=False)

### Categorical Correlations

In [ ]:
raw_data_encoded = pd.get_dummies(raw_data, columns=['furnishing_status'], drop_first=False)

raw_data_encoded[['furnishing_status_semi-furnished', 'furnishing_status_furnished','furnishing_status_unfurnished', 'price']].corr().sort_values(by='price',ascending=False)

### Feature Importance Heatmap

In [ ]:
correlation_matrix = raw_data_encoded.corr()

plt.figure(figsize=(10, 8)) 

sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')

plt.title('Correlation Heatmap')
plt.show()

## Correlation Analysis Conclusions:
- Features: area, number of bathrooms, number of stories, air conditioning, and parking slots have the strongest positive correlation with price.
- Features: unfurnished status is negatively correlated with price
- Features: semi-firnished, firnished, hot water heating, guestroom, and basement have the least effect on price

## Step 5: Visualize Data


In [ ]:
raw_data.head()
raw_data.columns

In [ ]:
all_columns = ['price', 'area', 'no_bedrooms', 'no_bathrooms', 'no_stories',
       'mainroad_connection', 'basement', 'guestroom', 'hot_water_heating',
       'air_conditioning', 'no_parking_slots', 'furnishing_status']

distribution_columns = ['price', 'area']

count_columns = ['no_bedrooms', 'no_bathrooms', 'no_stories', 'no_parking_slots']

categorical_columns = 'furnishing_status'

bool_columns = ['mainroad_connection', 'basement', 'guestroom', 'hot_water_heating','air_conditioning']

target='price'

In [ ]:
def distribution_plot(df,column):
    
    plt.figure(figsize=(8, 6))
    sns.histplot(df[column], bins=40, kde=True)

    plt.title(f'Distribution of {column.title()}', fontsize=16)
    plt.xlabel(column, fontsize=12)
    plt.ylabel('Frequency', fontsize=12)

    plt.show()

def boxplot(df,column,target):
    plt.figure(figsize=(8, 6))
    sns.boxplot(x=column, y=target, data=df,palette='Set1')

    plt.title(f'{column} Category Distribution', fontsize=16)
    plt.xlabel(column+' Categories', fontsize=12)
    plt.ylabel(target.title(), fontsize=12)
    plt.show()

def scatter_plot(df,xcol,ycol,hue=None):
    plt.figure(figsize=(8, 6))
    if hue == None:
        sns.scatterplot(x=xcol, y=ycol, data=df)
    else:
        sns.scatterplot(x=xcol, y=ycol, data=df, hue=hue, palette='Set1')

    plt.title(f'{xcol.title()} And {ycol.title()} Scatter Plot', fontsize=16)
    plt.xlabel(xcol.title(), fontsize=12)
    plt.ylabel(ycol.title(), fontsize=12)
    plt.show()

def pair_plot(df,columns):
    sns.pairplot(df[columns])
    plt.show()

def bar_plot(df,xcol,ycol):
    plt.figure(figsize=(8, 6))
    
    sns.barplot(x=xcol, y=ycol, data=df, palette='Set1')

    plt.xlabel(xcol.title(), fontsize=12)
    plt.ylabel(ycol.title(), fontsize=12)
    plt.show()

def count_plot(df,column):
    plt.figure(figsize=(8, 6))
    
    sns.countplot(data=df,x=column, palette='Set1').set_title(f"Count of {column.title()}")

    plt.xlabel(column.title(), fontsize=12)
    plt.ylabel("Count", fontsize=12)
    plt.show()

def violin_plot(df,xcol,ycol):
    plt.figure(figsize=(8, 6))
    
    sns.violinplot(x=xcol, y=ycol, data=raw_data_encoded)

    plt.xlabel(xcol.title(), fontsize=12)
    plt.ylabel(ycol.title(), fontsize=12)
    plt.show()   

In [ ]:
for col in distribution_columns:
    distribution_plot(raw_data,col)

In [ ]:
boxplot(raw_data,categorical_columns,target)

### Evidence of some outliers shown here

In [ ]:
scatter_plot(raw_data, 'area','price')

In [ ]:
pair_plot(raw_data,['price', 'area', 'no_bedrooms', 'no_bathrooms'])

In [ ]:
for cols in count_columns:
    count_plot(raw_data,cols)

### Uneven distribution shown here between feature frequencies

In [ ]:
sns.regplot(x='area',y='price', data=raw_data,scatter_kws={'color': 'blue'},line_kws={'color': 'red'}).set_title("Area Price Distribution")
plt.show()

In [ ]:
sns.regplot(x='no_bedrooms',y='area', data=raw_data,scatter_kws={'color': 'blue'},line_kws={'color': 'red'}).set_title("Area Bedroom Distribution")
plt.show()

## Step 6: Model Creation

### Initial Model will be standard multiple linear regression

In [ ]:
x_data = raw_data_encoded.drop('price', axis=1)
y_data = raw_data_encoded['price']

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.4, random_state=1)

print("number of test samples :", x_test.shape[0])
print("number of training samples:",x_train.shape[0])

In [ ]:
lm = LinearRegression()
lm.fit(x_train,y_train)
y_hat = lm.predict(x_test)

In [ ]:
print(r2_score(y_test,y_hat))

In [ ]:
plt.figure(figsize=(10, 6))
ax1 = sns.distplot(raw_data_encoded['price'], hist=False, color="r", label="Actual Value")
sns.distplot(y_hat, hist=False, color="b", label="Fitted Values" , ax=ax1)

plt.title('Actual vs Fitted Values for Yield')
plt.xlabel('House Price')
plt.ylabel('Density')
plt.legend()
plt.show()

### Initial model performs well but is not able to fully grasp the trend. The model is showing some signs of over fitting.

## Step 7: Model Evaluation and Refinement:

### Polynomial Feature Degree test:

In [ ]:
lr_test = LinearRegression()
R2_test = []
order = [1, 2, 3, 4, 5]
for n in order:
    pr = PolynomialFeatures(degree=n)
    x_train_pr = pr.fit_transform(x_train)
    x_test_pr = pr.fit_transform(x_test)    
    lr_test.fit(x_train_pr, y_train)
    R2_test.append(lr_test.score(x_test_pr, y_test))

In [ ]:
plt.plot(order, R2_test)
plt.xlabel('order')
plt.ylabel('R^2')
plt.title('R^2 Using Test Data') 
plt.show()

### Alpha value test:

In [ ]:
pr_test=PolynomialFeatures(degree=2)
x_train_pr=pr_test.fit_transform(x_train)
x_test_pr=pr_test.fit_transform(x_test)

In [ ]:
R2_test = []
R2_train = []
Alpha = np.arange(0.001,1,0.001)
pbar = tqdm(Alpha)

for alpha in pbar:
    RigeModel = Ridge(alpha=alpha) 
    RigeModel.fit(x_train_pr, y_train)
    test_score, train_score = RigeModel.score(x_test_pr, y_test), RigeModel.score(x_train_pr, y_train)
    pbar.set_postfix({"Test Score": test_score, "Train Score": train_score})
    R2_test.append(test_score)
    R2_train.append(train_score)

In [ ]:
plt.figure(figsize=(10, 6))  
plt.plot(Alpha, R2_test, label='validation data')
plt.plot(Alpha, R2_train, 'r', label='training Data')
plt.xlabel('alpha')
plt.ylabel('R^2')
plt.ylim(0, 1)
plt.legend()
plt.show()

### Grid Search Test:

In [ ]:
alpha_params = [{'alpha': [0.0001,0.001,0.01, 0.1, 1, 10]}]

In [ ]:
ridge_test=Ridge()
grid_test = GridSearchCV(ridge_test, alpha_params,cv=4)

In [ ]:
grid_test.fit(x_train, y_train)

In [ ]:
BestRR=grid_test.best_estimator_
print(BestRR)
print(BestRR.score(x_test, y_test))

### Model Creation With Optimal Parameters

In [ ]:
pr_opt = PolynomialFeatures(degree=2)
RidgeModelOPT = Ridge(alpha=1)

x_train_pr_opt = pr_opt.fit_transform(x_train)
x_test_pr_opt = pr_opt.fit_transform(x_test)

RidgeModelOPT.fit(x_train_pr_opt, y_train)
y_hat_opt = RidgeModelOPT.predict(x_test_pr_opt)


In [ ]:
print(r2_score(y_test,y_hat_opt))

In [ ]:
plt.figure(figsize=(8, 6))
sns.scatterplot(x=y_test, y=y_hat_opt, color='blue', label="Predicted vs Actual")

plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', lw=2, label="Ideal fit")

plt.title(f"Ridge Regression Predictions (R2 = {r2_score(y_test, y_hat_opt):.2f})", fontsize=16)
plt.xlabel("Actual Values", fontsize=12)
plt.ylabel("Predicted Values", fontsize=12)
plt.legend()

plt.show()

In [ ]:
plt.figure(figsize=(10, 6))

ax1 = sns.kdeplot(y_test, color="r", label="Actual Values", lw=2)

sns.kdeplot(y_hat_opt, color="b", label="Fitted Values", lw=2, ax=ax1)

plt.title('Actual vs Fitted Values for House Price', fontsize=16)
plt.xlabel('House Price', fontsize=12)
plt.ylabel('Density', fontsize=12)

plt.legend()
plt.show()

## Conclusions:
- Model shows less overfitting but still not grasping the trend
- r2 score remained around the same

## Next Steps:
- remove outliers
- scale down large numbers
- cross validation
- pipeline

### Filtering Outliers

In [ ]:
numeric_data = raw_data_encoded.select_dtypes(include=[np.number])

Q1 = numeric_data.quantile(0.25)
Q3 = numeric_data.quantile(0.75)

IQR = Q3 - Q1

filter = (numeric_data >= (Q1 - 1.5 * IQR)) & (numeric_data <= (Q3 + 1.5 * IQR))

filtered_data = raw_data_encoded[filter.all(axis=1)]

print(f"Original Data Shape: {raw_data_encoded.shape}")
print(f"Filtered Data Shape: {filtered_data.shape}")

### Pipeline Creation to Implement Scale and Streamline Process

In [ ]:
Input=[('scale',StandardScaler()), ('polynomial', PolynomialFeatures(degree=2,include_bias=False)), ('model',Ridge(alpha=10))]
pipe=Pipeline(Input)
z=filtered_data.drop('price',axis=1)
y = filtered_data['price']
pipe.fit(z,y)
ypipe=pipe.predict(z)

print('pipeline R2: ', r2_score(y, ypipe))

In [ ]:
cv_scores = cross_val_score(pipe, z, y, cv=5) 

print("Cross-Validation Scores:", cv_scores)
print("Average Cross-Validation Score:", cv_scores.mean())

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(y, ypipe, color='blue', label='Predicted vs Actual')
plt.plot([y.min(), y.max()], [y.min(), y.max()], color='red', linewidth=2, label='Ideal fit') 
plt.title('Actual vs Predicted Values')
plt.xlabel('Actual Values (Price)')
plt.ylabel('Predicted Values (Price)')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
sns.kdeplot(y, color='r', label='Actual Values', lw=2)
sns.kdeplot(ypipe, color='b', label='Predicted Values', lw=2)
plt.title('KDE Plot of Actual vs Predicted Values')
plt.xlabel('House Price')
plt.ylabel('Density')
plt.legend()
plt.show()

### Hyperparameter Tuning:

In [ ]:
param_grid = {
    'polynomial__degree': [1, 2, 3, 4],  
    'model__alpha': [0.001, 0.1, 1, 10]      
}

grid_search = GridSearchCV(pipe, param_grid, cv=5, scoring='r2')
grid_search.fit(z, y)

print("Best parameters found:", grid_search.best_params_)
print("Best cross-validation R2 score:", grid_search.best_score_)

## Conclusions:
- The initial pipeline performed well reaching a R2 of 0.716
- upon inspecion of cross validation scores it performed extremly poor, showing that it performs worse than a model which just guesses the mean

## Nexts Steps:
- Feature engineering
- Adjusting for multicollinearity between features
- Dropping unecessary features